In [1]:
# PYTHON BOOTSTRAPPING THE YIELD CURVE
# Copyright Sheikh Pancham 2021 sheikh.pancham@gmail.com
# Connect on LinkedIn: https://www.linkedin.com/in/sheikhpancham/
# I, Sheikh Pancham, legally own this product/code and all intellectual property because I wrote it, and am therefore 
# authorized to distribute it freely to the global public. Any party is also authorized to freely distribute this 
# product/code but not to sell it.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import *
from dateutil.relativedelta import *
from scipy import interpolate
from openpyxl import Workbook, load_workbook


class CountryHoliday:
    def __init__(self):
        pass
        
    def _IsHoliday_(self,date,calendar):
        weekdays = [0,1,2,3,4]
        if not date.weekday() in weekdays:
            return False
        
        y = date.year
        m = date.month
        d = date.day
        wkd = date.weekday()
       
        if calendar == 'NY':
            # January 1
            if (m==1 and d==1) or (m==12 and d==31 and wkd==4) or \
                (m==1 and d==2 and wkd==0):
                return True
            # Martin Luther King, third Monday of January
            if m==1 and wkd==0 and (d>14 and d<22):
                return True
            # Washington's Birthday, third Monday of February
            if m==2 and wkd==0 and (d>14 and d<22):
                return True
            # Memorial Day, last Monday of May
            if m==5 and wkd==0 and d>24:
                return True
            # Independence Day, July 4
            if (m==7 and d==4) or (m==7 and d==3 and wkd==4) or \
                (m==7 and d==5 and wkd==0):
                return True
            # Labor Day, the first Monday of September
            if m==9 and wkd==0 and d<8:
                return True
            # Columbus Day, second Monday of October
            if m==10 and wkd==0 and (d>7 and d<15):
                return True
            # Veterans Day, November 11
            if (m==11 and d==11) or (m==11 and d==10 and wkd==4) or \
                (m==11 and d==12 and wkd==0):
                return True
            # Thanksgiving Day, fourth Thursday of November
            if m==11 and wkd==3 and (d>21 and d<29):
                return True
            # Christmas Day, December 25
            if (m==12 and d==25) or (m==12 and d==24 and wkd==4) or \
                (m==12 and d==26 and wkd==0):
                return True
        return False
            
class CurveDate(CountryHoliday):
    def __init__(self):
        pass
    
    def _IsWeekend_(self,date):
        weekdays = [0,1,2,3,4]
        return date.weekday() not in weekdays
                       
    def _AddBusinessDays_(self,date,numdays,busdayconv,calendar):         
        next_date = date
        for i in range(numdays):
            next_date = next_date + relativedelta(days=+1)
            while (self._IsWeekend_(next_date)) or (self._IsHoliday_(next_date,calendar)):
                next_date = next_date + relativedelta(days=+1)
        return next_date
    
    def _AddBusinessMonths_(self,date,nummonths,busdayconv,calendar):
        next_date = date
#         for i in range(nummonths):   
        next_date = next_date + relativedelta(months=+nummonths)
        while (self._IsWeekend_(next_date)) or (self._IsHoliday_(next_date,calendar)):
            next_date = next_date + relativedelta(days=+1)
        return next_date
        
    def _AddBusinessYears_(self,date,numyears,busdayconv,calendar):
        next_date = date
#         for i in range(numyears):
        next_date = next_date + relativedelta(years=+numyears)
        while (self._IsWeekend_(next_date)) or (self._IsHoliday_(next_date,calendar)):
            next_date = next_date + relativedelta(days=+1)
        return next_date
    
    def _YFrac_(self, date1, date2, daycountconvention):
        if daycountconvention == 'ACTACT':
            pass
        elif daycountconvention == 'ACT365':
            # the difference between two datetime objects is a timedelta object
            delta = date2-date1
            delta_fraction = delta.days / 365.0
            return delta_fraction
        elif daycountconvention == 'ACT360':
            # the difference between two datetime objects is a timedelta object
            delta = date2-date1
            delta_fraction = delta.days / 360.0
            return delta_fraction
        elif daycountconvention == 'Thirty360':
            pass
        else:
            # the difference between two datetime objects is a timedelta object
            delta = date2-date1
            delta_fraction = delta.days / 360.0
            return delta_fraction
        
    
class YieldCurve(CurveDate,CountryHoliday):
    def __init__(self):
        self.dfcurve = pd.DataFrame() # initialize dataframe for curve data
    
    def __GetSwapCurveData__(self):
        filein = "swapcurvedata.xlsx"
        self.dfcurve = pd.read_excel(filein, sheet_name='swapcurve', index_col='Tenor')
        print(self.dfcurve)
        self.dfcurveparams = pd.read_excel(filein, sheet_name='curveparams')
        self.dfcurveparams.loc[self.dfcurveparams.index[0], 'Date'] = date.today()
        
        busdayconv = self.dfcurveparams['BusDayConv'].iloc[0]
        calendar = self.dfcurveparams['Calendar'].iloc[0]
        curvedate = self.dfcurveparams['Date'].iloc[0]
        while (self._IsWeekend_(curvedate) or self._IsHoliday_(curvedate,calendar)):
            curvedate = curvedate + relativedelta(days=+1)
        curvesettledays = self.dfcurveparams['SettleDays'].iloc[0]
        curvesettledate = self._AddBusinessDays_(curvedate,curvesettledays,busdayconv,calendar)
        self.dfcurveparams.loc[self.dfcurveparams.index[0], 'SettleDate'] = curvesettledate
        
    def _DatesForTenors_(self):
        curvesettledate = self.dfcurveparams['SettleDate'].iloc[0]
        busdayconv = self.dfcurveparams['BusDayConv'].iloc[0]
        calendar = self.dfcurveparams['Calendar'].iloc[0]
        
        self.dfcurve.loc[self.dfcurve.index=='ON', 'Date'] = \
                        self._AddBusinessDays_(curvesettledate,1,busdayconv,calendar)
        self.dfcurve.loc[self.dfcurve.index=='1W', 'Date'] = \
                        self._AddBusinessDays_(curvesettledate,5,busdayconv,calendar)
        for i in range (len(self.dfcurve)):
            if self.dfcurve.index[i][-1] == 'M':
                num = int(self.dfcurve.index[i][:-1])       
                self.dfcurve.loc[self.dfcurve.index[i],'Date'] = \
                        self._AddBusinessMonths_(curvesettledate,num,busdayconv,calendar)
            elif self.dfcurve.index[i][-1] == 'Y':
                num = int(self.dfcurve.index[i][:-1])       
                self.dfcurve.loc[self.dfcurve.index[i],'Date'] = \
                        self._AddBusinessYears_(curvesettledate,num,busdayconv,calendar)
                          
    def _YearFractionsForTenors_(self):
        curvesettledate = self.dfcurveparams['SettleDate'].iloc[0]
        for i in range(len(self.dfcurve)):
            daycntconv = self.dfcurve['Daycount'].iloc[i]
            if i == 0:
                self.dfcurve.loc[self.dfcurve.index[i],'YearFraction'] = \
                    self._YFrac_(curvesettledate, self.dfcurve['Date'].iloc[i], daycntconv)
            else:
                self.dfcurve.loc[self.dfcurve.index[i], 'YearFraction'] = \
                    self._YFrac_(self.dfcurve['Date'].iloc[i-1], self.dfcurve['Date'].iloc[i], daycntconv)
            self.dfcurve.loc[self.dfcurve.index[i], 'CumYearFraction'] = \
                self._YFrac_(curvesettledate, self.dfcurve['Date'].iloc[i], daycntconv)
           
            
    def _SwapYearFractions_(self,frequency,term):
        curvesettledate = self.dfcurveparams['SettleDate'].iloc[0]
        busdayconv = self.dfcurveparams['BusDayConv'].iloc[0]
        calendar = self.dfcurveparams['Calendar'].iloc[0]
        
        if frequency == 'S':
            period = 0.5
        elif frequency == 'Q':
            period = 0.25
        else:
            period = 0.5
            
        swap_months, swap_dates, swap_year_fractions = [],[],[]
        swap_months = [int(12 * period * i) for i in range(1,2*term)]
#         print(swap_months)
        
        for nummonths in swap_months:
            swap_dates.append(self._AddBusinessMonths_(curvesettledate,nummonths,busdayconv,calendar))
#         print(swap_dates)
        
        for swap_date in swap_dates:
            swap_year_fractions.append(self._YFrac_(curvesettledate,swap_date,'ACT/360'))
#         print(swap_year_fractions)
        return swap_year_fractions
               
    def _ZeroRates_(self):
        for i in range(len(self.dfcurve)):
            if self.dfcurve['Type'].iloc[i] == 'Deposit':
                self.dfcurve.loc[self.dfcurve.index[i],'ZeroRate'] = \
                        (1 / self.dfcurve['CumYearFraction'].iloc[i]) * \
                         np.log([1.0 + self.dfcurve['SwapRate'].iloc[i] * \
                                self.dfcurve['CumYearFraction'].iloc[i]])
            elif self.dfcurve['Type'].iloc[i] == 'EuroDollarFuture':
                rate_continuous = 0.0
                rate_continuous = 4 * np.log([1.0 + self.dfcurve['SwapRate'].iloc[i] * \
                                              self.dfcurve['YearFraction'].iloc[i]])
                self.dfcurve.loc[self.dfcurve.index[i], 'ZeroRate'] = \
                        (rate_continuous * self.dfcurve['YearFraction'].iloc[i] + \
                        self.dfcurve['ZeroRate'].iloc[i-1] * self.dfcurve['CumYearFraction'].iloc[i-1]) / \
                            self.dfcurve['CumYearFraction'].iloc[i]
            else:
                sumproduct = 0.0     
                    
                if self.dfcurve['Type'].iloc[i] == 'Swap':
                    frequency = self.dfcurve['Frequency'].iloc[i]
                    term = int(self.dfcurve.index[i][:-1])
                    swap_year_fractions = self._SwapYearFractions_(frequency,term)
                    # set up interpolation object
                    x = pd.Series(self.dfcurve['CumYearFraction'][:i])
                    y = pd.Series(self.dfcurve['ZeroRate'][:i])
                    zero_tck = interpolate.splrep(x,y)
                    
                for swap_yf in swap_year_fractions:
                    zero_rate = 0.0
                    zero_rate = interpolate.splev(swap_yf, zero_tck)
                    sumproduct = sumproduct + (self.dfcurve['SwapRate'].iloc[i] / 2.0) * \
                                 np.exp(-zero_rate * swap_yf)
               
                self.dfcurve.loc[self.dfcurve.index[i], 'ZeroRate'] = (-1 * np.log((1.0 - sumproduct) / \
                            (1.0 + self.dfcurve['SwapRate'].iloc[i] / 2.0))) / \
                            self.dfcurve['CumYearFraction'].iloc[i]
                
        fileout = "yieldcurve.xlsx"
        self.dfcurve.to_excel(fileout, sheet_name='yieldcurve', index=True)
                
        print(self.dfcurve.head(20))
                           
    
    def _DiscountFactors_(self):
        for i in range(len(self.dfcurve)):
            rate_i = self.dfcurve['SwapRate'].iloc[i]
            yearfraction_i = self.dfcurve['YearFraction'].iloc[i]
            cumyearfraction_i = self.dfcurve['CumYearFraction'].iloc[i]
            self.dfcurve.loc[self.dfcurve.index[i], 'DiscountFactor'] = np.exp(-rate_i * cumyearfraction_i)
            
    def _ForwardRates_(self):
        for i in range(len(self.dfcurve)):
            if i == 0:
                self.dfcurve.loc[self.dfcurve.index[i], 'ForwardRate'] = 0.0
            else:
                discountfactor_i = self.dfcurve['DiscountFactor'].iloc[i]
                discountfactor_iminusone = self.dfcurve['DiscountFactor'].iloc[i-1]
                yearfraction_i = self.dfcurve['YearFraction'].iloc[i]
                self.dfcurve.loc[self.dfcurve.index[i], 'ForwardRate'] = \
                            (discountfactor_iminusone / discountfactor_i - 1) / yearfraction_i
                
                
    def _PriceInterestRateSwap_(self):
        
        # using openpyxl functionality
        # calculate leg1 PV
        self.wb = load_workbook('swapcurvedata.xlsx')
        self.ws = self.wb['swap']
        self.leg1FxdFlt = self.ws['B2'].value
        self.leg1RateSpread = float(self.ws['B3'].value)
        self.leg1datesettle = self.ws['B4'].value
        self.leg1Freq = self.ws['B5'].value
        self.leg1Horizon = self.ws['B6'].value
        self.leg1Calendar = self.ws['B7'].value
        self.leg1BusDayConv = self.ws['B8'].value
        self.leg1DayCntConv = self.ws['B9'].value
        self.leg1LegNotional = float(self.ws['B10'].value)
        print(self.leg1LegNotional)
        
        iLeg1NumFreq = int(self.leg1Freq[:-1])
        iLeg1NumHorizon = int(self.leg1Horizon[:-1])
        
        leg1sPeriod = self.leg1Freq[-1] + self.leg1Horizon[-1]
        if leg1sPeriod == 'MM' or leg1sPeriod == 'YY':
            leg1NumPayments = iLeg1NumHorizon // iLeg1NumFreq
        elif leg1sPeriod == 'MY':
            leg1NumPayments = iLeg1NumHorizon * 12 // iLeg1NumFreq
        elif leg1sPeriod == 'YM':
            leg1NumPayments = iLeg1NumHorizon // (iLeg1NumFreq * 12)
        
        if self.leg1Freq == '6M':
            leg1YF = 0.5
        elif self.leg1Freq == '3M':
            leg1YF = 0.25
        
        x = pd.Series(self.dfcurve['CumYearFraction'])
        y = pd.Series(self.dfcurve['DiscountFactor'])
        df_tck = interpolate.splrep(x,y)
        
        leg1PV = 0.0
        for i in range(1,leg1NumPayments):
            leg1cyf_i = leg1YF * i
            leg1discountfactor_i = interpolate.splev(leg1cyf_i, df_tck)
            leg1cashflow_i = self.leg1LegNotional * self.leg1RateSpread * leg1YF * leg1discountfactor_i
            leg1PV = leg1PV + leg1cashflow_i
        print(leg1PV)
        
        # calculate leg2 PV
        self.leg2FxdFlt = self.ws['E2'].value
        self.leg2RateSpread = float(self.ws['E3'].value)
        self.leg2datesettle = self.ws['E4'].value
        self.leg2Freq = self.ws['E5'].value
        self.leg2Horizon = self.ws['E6'].value
        self.leg2Calendar = self.ws['E7'].value
        self.leg2BusDayConv = self.ws['E8'].value
        self.leg2DayCntConv = self.ws['E9'].value
        self.leg2LegNotional = float(self.ws['E10'].value)
        
        iLeg2NumFreq = int(self.leg2Freq[:-1])
        iLeg2NumHorizon = int(self.leg2Horizon[:-1])
        
        leg2sPeriod = self.leg2Freq[-1] + self.leg2Horizon[-1]
        if leg2sPeriod == 'MM' or leg2sPeriod == 'YY':
            leg2NumPayments = iLeg2NumHorizon // iLeg2NumFreq
        elif leg2sPeriod == 'MY':
            leg2NumPayments = iLeg2NumHorizon * 12 // iLeg2NumFreq
        elif leg2sPeriod == 'YM':
            leg2NumPayments = iLeg2NumHorizon // (iLeg2NumFreq * 12)
        
        if self.leg2Freq == '6M':
            leg2YF = 0.5
        elif self.leg2Freq == '3M':
            leg2YF = 0.25
        
        a = pd.Series(self.dfcurve['CumYearFraction'])
        b = pd.Series(self.dfcurve['ForwardRate'])
        fr_tck = interpolate.splrep(a,b)
        
        leg2PV = 0.0
        for i in range(1,leg2NumPayments):
            leg2cyf_i = leg2YF * i
            leg2discountfactor_i = interpolate.splev(leg2cyf_i, df_tck)
            leg2forwardrate_i = interpolate.splev(leg2cyf_i, fr_tck)
            leg2cashflow_i = self.leg2LegNotional * leg2forwardrate_i * leg2YF * leg2discountfactor_i
            leg2PV = leg2PV + leg2cashflow_i
        print(leg2PV)
        print(leg1PV + leg2PV)
            
        self.ws['H2'] = leg1PV
        self.ws['H3'] = leg2PV
        self.ws['H4'] = leg1PV + leg2PV
        self.wb.save('swapcurvedata.xlsx')
                
    def BootstrapYieldCurve(self):
        self.__GetSwapCurveData__()
        self._DatesForTenors_()
        self._YearFractionsForTenors_()
        self._ZeroRates_()
        self._DiscountFactors_()
        self._ForwardRates_()
        self._PriceInterestRateSwap_()
        print(self.dfcurve)
            
irs = YieldCurve()
irs.BootstrapYieldCurve()
     

                   Type    Frequency Daycount  SwapRate
Tenor                                                  
ON              Deposit  Zero Coupon   ACT360  0.000300
1W              Deposit  Zero Coupon   ACT360  0.000724
1M              Deposit  Zero Coupon   ACT360  0.001185
3M              Deposit  Zero Coupon   ACT360  0.001884
6M              Deposit  Zero Coupon   ACT360  0.002030
9M              Deposit  Zero Coupon   ACT360  0.002500
1Y              Deposit  Zero Coupon   ACT360  0.002838
15M    EuroDollarFuture            Q   ACT360  0.003181
18M    EuroDollarFuture            Q   ACT360  0.003525
21M    EuroDollarFuture            Q   ACT360  0.003583
2Y     EuroDollarFuture            Q   ACT360  0.003641
3Y     EuroDollarFuture            Q   ACT360  0.003797
4Y     EuroDollarFuture            Q   ACT360  0.004000
5Y                 Swap            S   ACT360  0.008230
7Y                 Swap            S   ACT360  0.011550
10Y                Swap            S   ACT360  0